In [1]:
def decorator(func):
    def wrap(*args,**kwargs):
        print(f'calling{func.__name__} with args:{args},kwargs:{kwargs}')
        result=func(*args,**kwargs)
        print(f'{func.__name__} returned:{result}')
        return result
    return wrap
@decorator
def multiply_numbers(x,y):
    return x*y
result= multiply_numbers(10,20)
print('Result:',result)

callingmultiply_numbers with args:(10, 20),kwargs:{}
multiply_numbers returned:200
Result: 200


In [4]:
import time
def measure_execution_time(func):
    def wrapper(*args,**kwargs):
        start_time=time.time()
        result=func(*args,**kwargs)
        end_time=time.time()
        execution_time=end_time-start_time
        print(f'Function {func.__name__} took {execution_time:.4f} seconds to execute')
        return result
    return wrapper
@measure_execution_time
def calculate_multiply(numbers):
    tot=1
    for x in numbers:
        tot*=x
    return tot
result= calculate_multiply([1,2,3,4,5])
print('Result:',result)

Function calculate_multiply took 0.0000 seconds to execute
Result: 120


In [6]:
def convert_to_data_type(data_type):
    def decorator(func):
        def wrapper(*args,**kwargs):
            result=func(*args,**kwargs)
            return data_type(result)
        return wrapper
    return decorator
@convert_to_data_type(int)
def add_numbers(x,y):
    return x+y
result= add_numbers(10,20)
print('Result:',result,type(result))
@convert_to_data_type(str)
def concatenate_strings(x,y):
    return x+y
result=concatenate_strings('Python','Decorator')
print('Result:',result,type(result))

Result: 30 <class 'int'>
Result: PythonDecorator <class 'str'>


In [2]:
def cache_result(func):
    cache={}
    def wrapper(*args,**kwargs):
        key=(*args,*kwargs.items())
        if key in cache:
            print('Retrieving result from cache...')
            return cache[key]
        result=func(*args,**kwargs)
        cache[key]=result
        return result
    return wrapper
@cache_result
def calculate_multiply(x,y):
    print('Calculating the product of two numbers...')
    return x*y
print(calculate_multiply(4, 5))  # Calculation is performed
print(calculate_multiply(4, 5))  # Result is retrieved from cache
print(calculate_multiply(5, 7))  # Calculation is performed
print(calculate_multiply(5, 7))  # Result is retrieved from cache
print(calculate_multiply(-3, 7))  # Calculation is performed
print(calculate_multiply(-3, 7))  # Result is retrieved from cache

Calculating the product of two numbers...
20
Retrieving result from cache...
20
Calculating the product of two numbers...
35
Retrieving result from cache...
35
Calculating the product of two numbers...
-21
Retrieving result from cache...
-21


In [3]:
def validate_arguments(condition):
    def decorato(func):
        def wrapper(*args,**kwargs):
            if condition(*args,**kwargs):
                return func(*args,**kwargs)
            else:
                raise ValueError('Invalid arguments passed to the function')
        return wrapper
    return decorato
@validate_arguments(lambda x: x>0)
def calculate_cube(x):
    return x**3
print(calculate_cube(5))
print(calculate_cube(-2))

125


ValueError: Invalid arguments passed to the function

In [9]:
import sqlite3
import time
def retry_on_faliure(max_retries,delay=1):
    def decorator(func):
        def wrapper(*args,**kwargs):
            for _ in range(max_retries):
                try:
                    result=func(*args,**kwargs)
                    return result
                except Exception as e:
                    print(f'Error occurred:{e}.Retrying....')
                    time.sleep(delay)
            raise Exception('Maximum retries exceeded. Function Failed.')
        return wrapper
    return decorator
@retry_on_faliure(max_retries=5,delay=1)
def connect_to_database():
    conn = sqlite3.connect("example.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    result = cursor.fetchall()
    cursor.close()
    conn.close()
    return result
try:
    data = connect_to_database()
    print("Data retrieved successfully:", data)
except Exception as e:
    print(f"Failed to connect to the database: {e}")

Error occurred:no such table: users.Retrying....
Error occurred:no such table: users.Retrying....
Error occurred:no such table: users.Retrying....
Error occurred:no such table: users.Retrying....
Error occurred:no such table: users.Retrying....
Failed to connect to the database: Maximum retries exceeded. Function Failed.


In [11]:
import time
def rate_limits(max_calls, period):
    def decorator(func):
        calls = 0
        last_reset = time.time()
        def wrapper(*args, **kwargs):
            nonlocal calls, last_reset
            elapsed = time.time() - last_reset
            if elapsed > period:
                calls = 0
                last_reset = time.time()
            if calls >= max_calls:
                raise Exception("Rate limit exceeded. Please try again later.")
            calls += 1
            return func(*args, **kwargs)
        return wrapper
    return decorator
@rate_limits(max_calls=6, period=10)
def api_call():
    print("API call executed successfully...")
for _ in range(8):
    try:
        api_call()
    except Exception as e:
        print(f"Error occurred: {e}")

API call executed successfully...
API call executed successfully...
API call executed successfully...
API call executed successfully...
API call executed successfully...
API call executed successfully...
Error occurred: Rate limit exceeded. Please try again later.
Error occurred: Rate limit exceeded. Please try again later.


In [12]:
def add_logging(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned: {result}")
        return result
    return wrapper
@add_logging
def add_numbers(x, y):
    return x + y
result = add_numbers(200, 300)
print("Result:", result)

Calling add_numbers with args: (200, 300), kwargs: {}
add_numbers returned: 500
Result: 500


In [15]:
def handle_exceptions(default_response):
    def decorator(func):
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                print(f"Exception occurred: {e}")
                return default_response
        return wrapper
    return decorator
@handle_exceptions(default_response="An error occurred!")
def divide_numbers(x, y):
    return x / y
result = divide_numbers(7, 0)  # This will raise a ZeroDivisionError
print("Result:", result)

Exception occurred: division by zero
Result: An error occurred!


In [17]:
import inspect
def enforce_type_checking(func):
    def wrapper(*args, **kwargs):
        signature = inspect.signature(func)
        parameters = signature.parameters
        for i, arg in enumerate(args):
            param_name = list(parameters.keys())[i]
            param_type = parameters[param_name].annotation
            if not isinstance(arg, param_type):
                raise TypeError(f"Argument '{param_name}' must be of type '{param_type.__name__}'")
        for param_name, arg in kwargs.items():
            param_type = parameters[param_name].annotation
            if not isinstance(arg, param_type):
                raise TypeError(f"Argument '{param_name}' must be of type '{param_type.__name__}'")
        return func(*args, **kwargs)
    return wrapper
@enforce_type_checking
def multiply_numbers(x: int, y: int) -> int:
    return x * y
result = multiply_numbers(5, 7)  # No type errors, returns 30
print("Result:", result)
result = multiply_numbers("5", 7)  # Type error: 'x' must be of type 'int'

Result: 35


TypeError: Argument 'x' must be of type 'int'

In [18]:
import tracemalloc
def measure_memory_usage(func):
    def wrapper(*args, **kwargs):
        tracemalloc.start()
        result = func(*args, **kwargs)
        snapshot = tracemalloc.take_snapshot()
        top_stats = snapshot.statistics("lineno")
        print(f"Memory usage of {func.__name__}:")
        for stat in top_stats[:5]:
            print(stat)
        return result
    return wrapper
@measure_memory_usage
def calculate_factorial(n):
    if n == 0:
        return 1
    else:
        return n * calculate_factorial(n - 1)
result = calculate_factorial(5)
print("Factorial:", result)

Memory usage of calculate_factorial:
Memory usage of calculate_factorial:
C:\Users\panka\anaconda3\Lib\selectors.py:314: size=144 KiB, count=3, average=48.0 KiB
C:\Users\panka\anaconda3\Lib\asyncio\base_events.py:744: size=112 B, count=1, average=112 B
C:\Users\panka\AppData\Roaming\Python\Python311\site-packages\zmq\sugar\attrsettr.py:44: size=110 B, count=2, average=55 B
C:\Users\panka\AppData\Local\Temp\ipykernel_8136\3102669201.py:8: size=85 B, count=1, average=85 B
C:\Users\panka\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py:231: size=80 B, count=1, average=80 B
Memory usage of calculate_factorial:
C:\Users\panka\anaconda3\Lib\selectors.py:314: size=144 KiB, count=3, average=48.0 KiB
C:\Users\panka\anaconda3\Lib\tracemalloc.py:59: size=769 B, count=5, average=154 B
C:\Users\panka\AppData\Local\Temp\ipykernel_8136\3102669201.py:8: size=170 B, count=2, average=85 B
C:\Users\panka\AppData\Roaming\Python\Python311\site-packages\ipykernel\iostream.py:563: s

In [19]:
import time
def cache_with_expiry(expiry_time):
    def decorator(func):
        cache = {}
        def wrapper(*args, **kwargs):
            key = (*args, *kwargs.items())
            if key in cache:
                value, timestamp = cache[key]
                if time.time() - timestamp < expiry_time:
                    print("Retrieving result from cache...")
                    return value
            result = func(*args, **kwargs)
            cache[key] = (result, time.time())
            return result
        return wrapper
    return decorator
@cache_with_expiry(expiry_time=5)  # Cache expiry time set to 5 seconds
def calculate_multiply(x, y):
    print("Calculating product of two numbers...")
    return x * y
print(calculate_multiply(23, 5))  # Calculation is performed
print(calculate_multiply(23, 5))  # Result is retrieved from cache
time.sleep(5)
print(calculate_multiply(23, 5))  # Calculation is performed (cache expired)

Calculating product of two numbers...
115
Retrieving result from cache...
115
Calculating product of two numbers...
115
